In [28]:
import pandas as pd
from sklearn import preprocessing

pd.set_option('display.max_colwidth', None)

import os

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [29]:
# Min-Max normalization
# Create a minimum and maximum processor object,
min_max_scaler = preprocessing.MinMaxScaler()

# Min-max normalize column  and concat with
# original dataframe
def normalize_column(column, df):
    x = df[[column]].values.astype(float)
    x = x.reshape(-1, 1)
    x_scaled = min_max_scaler.fit_transform(x)
    norm_df = pd.DataFrame(x_scaled)
    norm_df.columns = [column + "_normalized"]
    
    norm_df = norm_df.reset_index()
    df = df.reset_index()
    dataframe = df.merge(norm_df, left_index = True, right_index = True) 
    
    return dataframe

In [30]:
line_folder_1 = "/Volumes/TOB_WD2/Microscopy/20200821_EB1/Lines_EB1_Half"
line_folder_2 = "/Volumes/TOB_WD2/Microscopy/20200828_EB1/Lines_EB1_Half"
line_folder_3 = "/Volumes/TOB_WD2/Microscopy/20200918_EB1/Lines_EB1_Half"
line_folder_4 = "/Volumes/TOB_WD2/Microscopy/20220916_EB1/Lines_EB1_Half"
line_folder_5 = "/Volumes/TOB_WD2/Microscopy/20220921_EB1/Lines_EB1_Half"
line_folder_6 = "/Volumes/TOB_WD2/Microscopy/20220922_EB1/Lines_EB1_Half"
line_folder_7 = "/Volumes/TOB_WD2/Microscopy/20221021_EB1/Lines_EB1_Half"
line_folder_8 = "/Volumes/TOB_WD2/Microscopy/20221027_EB1/Lines_EB1_Half"
line_folder_9 = "/Volumes/TOB_WD2/Microscopy/20221103_EB1/Lines_EB1_Half"
line_folder_10 = "/Volumes/TOB_WD2/Microscopy/20230728_EB1/Lines_EB1_Half" #CCB02 experiment
line_folder_11 = "/Volumes/TOB_WD2/Microscopy/20230809_EB1/Lines_EB1_Half" #CCB02 experiment 
line_folder_12 = "/Volumes/TOB_WD2/Microscopy/20231103_EB1_Osmotic/Lines_EB1_Half" #Hypotonic experiment 1 
line_folder_13 = "/Volumes/TOB_WD2/Microscopy/20231106_EB1_Osmotic/Lines_EB1_Half" #Hypotonic experiment 1 

def linescans(folder):
    dataframes = []
    max_dataframes = []
    for subdir, dirs, files in os.walk(folder):
        for file in files:
            if not file.startswith("."):
                filepath = subdir + os.sep + file
                if filepath.endswith(".txt"):
                    if file.startswith("20200821"):
                        cell_id = file.rsplit("-", 1)[0]
                    #elif file.startswith("20200828") or file.startswith("20200918"):
                    #    cell_id = file.rsplit("_", 1)[0]
                    else:
                        cell_id = file.rsplit("_", 1)[0]
                    
                    single_df = pd.read_csv(filepath, sep = " ", delimiter = "\t", encoding = 'unicode_escape')
                    single_df["Cell_ID"] = cell_id
                    single_df = single_df.rename(columns = {'Value': 'EB1'})
                    single_df.columns.values[0] = 'Distance'
                    
                    
                    # normalize staining
                    EB1_norm = normalize_column("EB1", single_df)
                    # normalize distance
                    D_norm = normalize_column("Distance", EB1_norm)
                   #D_norm["Distance_normalized"] = D_norm.Distance_normalized - 0.5
                    
                    D_norm = D_norm.loc[:,~D_norm.columns.str.startswith('index')]
                    dataframes.append(D_norm)
                    
                    Maximum = D_norm.query('EB1_normalized == EB1_normalized.max()')["Distance_normalized"].item()
                    
                    max_dict = {"Cell_ID": cell_id, "Distance_EB1_maximum": Maximum}
                    max_df = pd.DataFrame(max_dict, index = [0])
                    max_dataframes.append(max_df)
    
    lines = pd.concat(dataframes)
    maximums = pd.concat(max_dataframes)
    return lines, maximums

df1, max1 = linescans(line_folder_1)
df2, max2 = linescans(line_folder_2)
df3, max3 = linescans(line_folder_3)
df4, max4 = linescans(line_folder_4)
df5, max5 = linescans(line_folder_5)
df6, max6 = linescans(line_folder_6)
df7, max7 = linescans(line_folder_7)
df8, max8 = linescans(line_folder_8)
df9, max9 = linescans(line_folder_9)
df10, max10 = linescans(line_folder_10)
df11, max11 = linescans(line_folder_11)
df12, max12 = linescans(line_folder_12)
df13, max13 = linescans(line_folder_13)

In [31]:
df = pd.concat(
    [
        df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13
    ]
)

In [32]:
max_df = pd.concat(
    [
        max1, max2, max3, max4, max5, max6, max7, max8, max9, max10, max11, max12, max13
    ]
).reset_index()
max_df

index                                     Cell_ID  Distance_EB1_maximum
0        0  20200821_R1E309_EB1tdTomato_lipo075_01_003              0.380000
1        0  20200821_R1E309_EB1tdTomato_lipo075_01_006              0.324324
2        0  20200821_R1E309_EB1tdTomato_lipo075_01_011              0.261905
3        0  20200821_R1E309_EB1tdTomato_lipo075_01_011              0.416667
4        0  20200821_R1E309_EB1tdTomato_lipo075_01_014              0.489362
..     ...                                         ...                   ...
467      0       20231106_R1E309_EB1_11_Hypo7525_030-1              0.161290
468      0       20231106_R1E309_EB1_11_Hypo7525_032-1              0.125000
469      0       20231106_R1E309_EB1_11_Hypo7525_033-1              0.233333
470      0       20231106_R1E309_EB1_11_Hypo7525_034-1              0.172414
471      0       20231106_R1E309_EB1_11_Hypo7525_035-1              0.250000

[472 rows x 3 columns]

In [33]:
def get_condition(x):
    experiment = x.Experiment
    if experiment == 20200821 or experiment == 20200828 or experiment == 20200918 or experiment == 20200916:
        return "FBS+LIF"
    elif experiment == 20230728 or experiment == 20230809 or experiment == 20231103 or experiment == 20231106:
        condition_string = x.Cell_ID.split("_")[4]
        return condition_string
    else:
        condition_string = x.Cell_ID.split("_")[4]
        if condition_string == "Full":
            return "FBS+LIF"
        else:
            return "N2B27+RA"

In [34]:
df["Experiment"] = df.Cell_ID.str.split("_").str.get(0).astype(int)
df["Condition"] = df.apply(get_condition, axis = 1)
df["Incubation_h"] = 48

max_df["Experiment"] = max_df.Cell_ID.str.split("_").str.get(0).astype(int)
max_df["Condition"] = max_df.apply(get_condition, axis = 1)
max_df["Incubation_h"] = 48

In [35]:
outpath = "/Volumes/TOB_WD2/Image_Analysis/EB1/Dataframes" + "/" + "LineDataFrame.csv" 
df.to_csv(outpath)

print("Successfully exported data to " + outpath)

Successfully exported data to /Volumes/TOB_WD2/Image_Analysis/EB1/Dataframes/LineDataFrame.csv


In [36]:
maxoutpath = "/Volumes/TOB_WD2/Image_Analysis/EB1/Dataframes" + "/" + "MaximumDataFrame.csv" 
max_df.to_csv(maxoutpath)

print("Successfully exported dataframe with distance of eb1 maxima to " + maxoutpath)

Successfully exported dataframe with distance of eb1 maxima to /Volumes/TOB_WD2/Image_Analysis/EB1/Dataframes/MaximumDataFrame.csv
